# Improving on cleaning the original Datasets
Cleaning up the Data from each quarterly report with respect to its stock ticker.

### Importing Libraries and Data

In [1]:
import pandas as pd
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 70)
import numpy as np
from tqdm import tqdm_notebook as tqdm
import _pickle as pickle

In [2]:
with open("stockpup.pkl",'rb') as fp:
    stocks_df = pickle.load(fp)

### Setting the Index to the Date

In [3]:
def setting_index(df):
    """
    Returns a sorted date time index
    """
    df['Quarter end'] = pd.to_datetime(df['Quarter end'])
    df.set_index("Quarter end", inplace=True)
    return df.sort_index(ascending=True)

for i in tqdm(stocks_df.keys()):
    stocks_df[i] = setting_index(stocks_df[i])

### Replacing all "None" values with 0

In [4]:
for i in tqdm(stocks_df.keys()):
    stocks_df[i].replace("None", 0, inplace=True)

### Converting all values to numeric values

In [5]:
def to_num(stocks_df):  
    """
    Converts each column in the DFs in the dictionary into numerical values
    """
    num_df = {}

    for i in tqdm(stocks_df.keys()):
        # Creates a new DF
        n_df = pd.DataFrame(index=stocks_df[i].index, columns=stocks_df[i].columns)

        cols = n_df.columns
        
        # Iterate through each column
        for c in cols:
            # Converting each column's values to numerical values
            n_df[c] = pd.to_numeric(stocks_df[i][c])
            
        # Assigning the DF to the dictionary
        num_df[i] = n_df
    
    return num_df


num_df = to_num(stocks_df)

## Preprocessing the Data
Replacing values with percent difference between each quarter

Also:
 - Replacing infinite values with 1 or -1
 - Replacing NaNs with 0

In [10]:
def percent_improvement(num_df):
    """
    Shows the percent change from the previous to the current quarter
    """
    pcnt_df = {}
    
    for i in tqdm(num_df.keys()):
        # Creating a new DF
        df = pd.DataFrame(index=num_df[i].index, columns=num_df[i].columns)
        
        # Iterating through the DF columns
        for c in df.columns:
            if c == 'Price high':
                df[c] = num_df[i][c]
                
                # Another column to show the quarterly return it will have (for EDA purposes and class assignment only)
                df['Future % High'] = (num_df[i][c].pct_change(1)*100).shift(-1)
                
            elif c == 'Price low':
                df[c] = num_df[i][c]
                
                # Another column to show the quarterly return it will have
                df['Future % Low'] = (num_df[i][c].pct_change(1)*100).shift(-1)
            else:
                # Assigning the percent values to the new DF columns
                df[c] = num_df[i][c].pct_change(1)*100
            
        # Assigning the new DF to the dictionary and replacing infinite values with 1 and -1
        # Also handling the NaN values
        pcnt_df[i] = df.replace([np.inf, -np.inf], [1,-1]).dropna(1, 'all').fillna(0)
        
    return pcnt_df


pcnt_df = percent_improvement(num_df)

In [56]:
# def percent_improvement(num_df):
#     """
#     Shows the percent change from the previous to the current quarter
#     """
#     pcnt_df = {}
    
#     for i in tqdm(num_df.keys()):
#         # Creating a new DF
#         df = pd.DataFrame(index=num_df[i].index, columns=num_df[i].columns)
        
#         # Iterating through the DF columns
#         for c in df.columns:
#             if c == 'Price':
#                 df[c] = num_df[i][c]
                
#                 # Another column to show the quarterly return it will have (for EDA purposes and class assignment only)
#                 df['Future % Return'] = (num_df[i][c].pct_change(1)*100).shift(-1)
#             else:
#                 # Assigning the percent values to the new DF columns
#                 df[c] = num_df[i][c].pct_change(1)*100
            
#         # Assigning the new DF to the dictionary and replacing infinite values with 1 and -1
#         # Also handling the NaN values
#         pcnt_df[i] = df.replace([np.inf, -np.inf], [1,-1]).dropna(1, 'all').fillna(0)
        
#     return pcnt_df


# pcnt_df = percent_improvement(num_df)

## Creating the Classes
- Buy (because the highest high and lowest low of the quarter will both increase by 5% or more)
- Sell (because the lowest low and highest high of the quarter will both decrease by 5% or more)
- Hold (because it will not do either)

In [18]:
def price_check(df):
    """
    Creates classes of buy(1), hold(2), sell(0)
    """

    # Iterate through the dictionary of DFs
    for i in tqdm(df.keys()):
        # Create an empty list for classes
        dec = []
        
        # Iterating through the values of the 'Price' column
        for n in range(len(df[i].Price)):
            if df[i]['Future % High'][n]>=5 and df[i]['Future % Low'][n]>=5:
                # BUY
                dec.append(1)
            elif df[i]['Future % Low'][n]<=-5 and df[i]['Future % High'][n]<=-5:
                # SELL
                dec.append(0)
            else:
                # HOLD
                dec.append(2)
                
        # Assigning the list to new column for each DF
        df[i]['Decision'] = dec
        
    return df
                
         
new_df = price_check(pcnt_df)

### Excluding the first and last rows
This is done because the last row has no data to compare percent improvements to and the first row does not have any data to show if the price will increase in the future.

In [19]:
for i in tqdm(pcnt_df.keys()):
    new_df[i] = new_df[i][1:-1]

#### Checking an example stock's DF for correct labeling and returns

In [21]:
new_df['A'][['Price high', 'Price low', 'Future % High', 'Future % Low', 'Decision']].sort_index(ascending=False).head(10)

,Price high,Price low,Future % High,Future % Low,Decision
Quarter end,,,,,
2019-01-31,76.31,62.00,7.810248,18.967742,1
2018-10-31,72.37,61.01,5.444245,1.622685,2
2018-07-31,70.45,60.42,2.725337,0.976498,2
2018-04-30,74.99,63.38,-6.054141,-4.670243,2
2018-01-31,75.00,65.60,-0.013333,-3.384146,2
2017-10-31,68.52,58.22,9.457093,12.676056,1
2017-07-31,61.84,55.36,10.802070,5.166185,1
2017-04-30,55.99,48.47,10.448294,14.214978,1
2017-01-31,49.48,42.92,13.156831,12.931034,1


"Future % " will only be used for EDA/Labeling and will be removed during the modeling process.

### Combining all stock DFs into one

In [22]:
big_df = pd.DataFrame()

for i in tqdm(pcnt_df.keys()):
    big_df = big_df.append(new_df[i], sort=False)

#### Quick check for NaN values

In [23]:
big_df.isna().sum()

Shares                                            0
Shares split adjusted                             0
Split factor                                      0
Assets                                            0
Current Assets                                 9103
Liabilities                                       0
Current Liabilities                            9103
Shareholders equity                               0
Non-controlling interest                      13870
Goodwill & intangibles                         2693
Long-term debt                                 1705
Revenue                                           0
Earnings                                          0
Earnings available for common stockholders        0
EPS basic                                         0
EPS diluted                                       0
Dividend per share                             8469
Cash from operating activities                    0
Cash from investing activities                    0
Cash from fi

#### Filling the NaNs with 0

In [24]:
big_df.fillna(0, inplace=True)

In [25]:
# Checking the DF again for NaN
big_df.isna().sum()

Shares                                        0
Shares split adjusted                         0
Split factor                                  0
Assets                                        0
Current Assets                                0
Liabilities                                   0
Current Liabilities                           0
Shareholders equity                           0
Non-controlling interest                      0
Goodwill & intangibles                        0
Long-term debt                                0
Revenue                                       0
Earnings                                      0
Earnings available for common stockholders    0
EPS basic                                     0
EPS diluted                                   0
Dividend per share                            0
Cash from operating activities                0
Cash from investing activities                0
Cash from financing activities                0
Cash change during period               

### Excluding columns with less than 500 unique values
Except our class 'price_will_increase?' column.

In [26]:
big_df.nunique()<500

Shares                                        False
Shares split adjusted                         False
Split factor                                   True
Assets                                        False
Current Assets                                False
Liabilities                                   False
Current Liabilities                           False
Shareholders equity                           False
Non-controlling interest                      False
Goodwill & intangibles                        False
Long-term debt                                False
Revenue                                       False
Earnings                                      False
Earnings available for common stockholders    False
EPS basic                                     False
EPS diluted                                   False
Dividend per share                            False
Cash from operating activities                False
Cash from investing activities                False
Cash from fi

In [27]:
# Iterating through each column and checking if unique values are less than 500
# Then dropping those columns
for i in tqdm(big_df.columns):
    if big_df[i].nunique() < 500 and i != 'Decision':
        big_df.drop(i, axis=1, inplace=True)

#### Resetting the index

In [28]:
big_df.reset_index(drop=True, inplace=True)
big_df.head()

,Shares,Shares split adjusted,Assets,Current Assets,Liabilities,Current Liabilities,Shareholders equity,Non-controlling interest,Goodwill & intangibles,Long-term debt,Revenue,Earnings,Earnings available for common stockholders,EPS basic,EPS diluted,Dividend per share,Cash from operating activities,Cash from investing activities,Cash from financing activities,Cash change during period,Cash at end of period,Capital expenditures,Price,Price high,Price low,ROE,ROA,Book value of equity per share,P/B ratio,P/E ratio,Cumulative dividends per share,Dividend payout ratio,Long-term debt to equity ratio,Equity to assets ratio,Net margin,Asset turnover,Free cash flow per share,Current ratio,Future % High,Future % Low,Decision,Preferred equity
0,0.060170,0.060170,3.011116,1.505420,2.212896,5.757136,3.477486,0.0,0.000000,0.0,10.641140,26.717557,26.717557,23.333333,20.000000,0.0,-109.895833,63.942308,-100.922819,-128.508772,-28.508772,216.666667,92.855945,159.00,71.00,0.000000,0.0,3.427419,1.000000,0.000000,0.0,0.0,0.0,0.459442,0.000000,0.00000,-137.179487,-4.020246,-36.635220,-42.605634,0,0.0
1,0.164196,0.164196,6.911624,5.675302,9.182531,14.821592,5.601034,0.0,0.000000,0.0,7.444668,-6.626506,-6.626506,-8.108108,-5.555556,0.0,-281.578947,22.287390,-763.636364,-29.487179,-28.118609,133.684211,-38.478261,100.75,40.75,0.000000,0.0,5.458090,-40.465919,0.000000,0.0,0.0,0.0,-1.230090,0.000000,0.00000,17.241379,-7.966629,-37.468983,-4.760736,2,0.0
2,0.739888,0.739888,7.640220,5.819611,8.034188,9.880478,7.405141,0.0,1.000000,0.0,26.292135,96.774194,96.774194,97.058824,91.176471,0.0,513.043478,-63.788969,-71.232877,-206.545455,41.678521,59.909910,-28.042403,63.00,38.81,0.000000,0.0,6.654344,-31.739130,0.000000,0.0,0.0,0.0,-0.223535,0.000000,0.00000,-144.117647,-3.696398,7.936508,-1.932492,2,0.0
3,0.088384,0.088384,9.293769,-3.430592,16.044304,11.312545,5.242165,0.0,117.773788,0.0,-15.747331,-49.508197,-49.508197,-49.253731,-49.230769,0.0,-146.572104,476.158940,2300.000000,-292.150171,-56.526104,-67.887324,4.164211,68.00,38.06,1.000000,0.0,5.112652,-2.335456,1.000000,0.0,0.0,0.0,-3.696591,1.000000,1.00000,-553.333333,-13.246196,-19.117647,-34.314241,0,0.0
4,0.066338,0.066338,-1.390096,-8.478301,-5.535860,0.912052,1.353546,0.0,2.061006,0.0,-16.156283,-37.662338,-37.662338,-38.235294,-36.363636,0.0,-100.000000,-100.000000,-100.000000,-166.785080,86.836028,199.122807,-24.570998,55.00,25.00,-13.111546,0.0,1.319044,-28.260870,-25.917859,0.0,0.0,0.0,2.775008,-8.163265,-6.47482,-100.000000,-9.304025,-25.127273,4.800000,2,0.0


### Counting how many classes there are
Must make sure they are equal for future modeling.

In [29]:
big_df['Decision'].value_counts()

2    33629
1    17554
0     8914
Name: Decision, dtype: int64

The 'Sell' [0] Class has the least so each class has to match that count

In [30]:
# Determining how many rows to remove
buy = len(big_df[big_df['Decision']==1])
hold = len(big_df[big_df['Decision']==2])
sell = len(big_df[big_df['Decision']==0])

rem_buy = buy - sell
rem_hold = hold - sell

#### Removing the excess values

In [31]:
import random

# Selecting only the 'Buy' rows
buy_df = big_df[big_df['Decision']==1]

# Selecting only the 'Hold' rows
hold_df = big_df[big_df['Decision']==2]

# Randomly choosing rows
drop_buy = random.sample(list(buy_df.index), rem_buy)
drop_hold = random.sample(list(hold_df.index), rem_hold)

# Dropping rows
buy_df.drop(drop_buy, inplace=True)
hold_df.drop(drop_hold, inplace=True)

/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


### Creating the DataFrame with classes equal

In [32]:
# First assigning the DF full of false values
sell_df = big_df[big_df['Decision']==0]

# Concatenating all
final_df = pd.concat([buy_df, hold_df, sell_df])

# Double checking to see if both values are the same
final_df['Decision'].value_counts()

2    8914
1    8914
0    8914
Name: Decision, dtype: int64

### Exporting the final dataframe

In [33]:
with open("main_df.pkl", 'wb') as fp:
    pickle.dump(final_df, fp)